# VacationPy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import api_key

### Import random city data

In [5]:
data = os.path.join(os.getcwd(), '..', 'WeatherPy','output','city_weather.csv')
df = pd.read_csv(data, index_col=0)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kariba,-16.5167,28.8000,80.20,71,20,5.32,ZW,1616421723
1,Albany,42.6001,-73.9662,51.01,36,0,2.13,US,1616421666
2,Norman Wells,65.2820,-126.8329,-22.00,76,40,2.30,CA,1616421649
3,Kushmurun,52.4503,64.6311,16.48,91,17,8.05,KZ,1616421724
4,Hermanus,-34.4187,19.2345,77.00,59,0,8.99,ZA,1616421673


### Humidity Heatmap

In [49]:
gmaps.configure(api_key=api_key)
locations = df[['Lat','Lng']]
weights = df['Humidity']
fig = gmaps.figure(center=(0,0),zoom_level=1.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame of ideal cities based on following weather criteria
* Max temperature lower than 80 degrees but higher than 70 degrees
* Wind speed less than 10 mph
* 0% cloudiness

In [39]:
ideal_df = df.loc[(df['Cloudiness'] == 0) & (df['Wind Speed'] < 10) & (df['Max Temp'] > 70) & (df['Max Temp'] < 80)]
ideal_df = ideal_df.dropna(how='any')
ideal_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Hermanus,-34.4187,19.2345,77.00,59,0,8.99,ZA,1616421673
15,São Filipe,14.8961,-24.4956,70.63,75,0,8.75,CV,1616421727
22,Port Elizabeth,-33.9180,25.5701,71.60,49,0,8.05,ZA,1616421729
41,Rikitea,-23.1203,-134.9692,79.65,71,0,6.22,PF,1616421734
98,Taltal,-25.4000,-70.4833,70.45,68,0,3.40,CL,1616421672
135,Santiago,-33.4569,-70.6483,72.00,55,0,4.61,CL,1616421685
234,Sur,22.5667,59.5289,77.74,64,0,7.74,OM,1616421655
372,Tura,25.5198,90.2201,76.06,20,0,2.21,IN,1616421836
431,Port Said,31.2565,32.2841,75.20,46,0,2.30,EG,1616421852
500,Srīvardhan,18.0333,73.0167,79.88,69,0,8.81,IN,1616421872


### Find nearest hotel to each ideal city

In [47]:
hotels = []
for index, row in ideal_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    try:
        res = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=lodging&key={api_key}')
        hotel = res.json()['results'][0]['name']
        hotels.append(hotel)
        print(f"Found hotel near {row['City']}: {hotel}")
    except:
        print(f"No hotels found near {row['City']}.")
        hotels.append('No hotels nearby')

Found hotel near Hermanus: Misty Waves Boutique Hotel
Found hotel near São Filipe: Tortuga B&B
Found hotel near Port Elizabeth: 39 On Nile Guest House
Found hotel near Rikitea: Pension Maro'i
Found hotel near Taltal: Residencial Karime
Found hotel near Santiago: Che Lagarto Hostel Santiago
Found hotel near Sur: Sur Plaza Hotel
Found hotel near Tura: Hotel RIKMAN Continental
Found hotel near Port Said: Aracan Hotel
Found hotel near Srīvardhan: Savli Resort/Hotel - Shrivardhan


In [41]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Hermanus,-34.4187,19.2345,77.00,59,0,8.99,ZA,1616421673,Misty Waves Boutique Hotel
15,São Filipe,14.8961,-24.4956,70.63,75,0,8.75,CV,1616421727,Tortuga B&B
22,Port Elizabeth,-33.9180,25.5701,71.60,49,0,8.05,ZA,1616421729,39 On Nile Guest House
41,Rikitea,-23.1203,-134.9692,79.65,71,0,6.22,PF,1616421734,Pension Maro'i
98,Taltal,-25.4000,-70.4833,70.45,68,0,3.40,CL,1616421672,Residencial Karime
135,Santiago,-33.4569,-70.6483,72.00,55,0,4.61,CL,1616421685,Che Lagarto Hostel Santiago
234,Sur,22.5667,59.5289,77.74,64,0,7.74,OM,1616421655,Sur Plaza Hotel
372,Tura,25.5198,90.2201,76.06,20,0,2.21,IN,1616421836,Hotel RIKMAN Continental
431,Port Said,31.2565,32.2841,75.20,46,0,2.30,EG,1616421852,Aracan Hotel
500,Srīvardhan,18.0333,73.0167,79.88,69,0,8.81,IN,1616421872,Savli Resort/Hotel - Shrivardhan


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

### Add markers for each hotel to heatmap

In [53]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))